This notebook simply uses matching if a dataset is in the document, it "predicts" the title.  It uses the 180 dataset list from the train data and adds some hand curated govt dataset titles.

In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from fuzzywuzzy import fuzz

In [ ]:
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

sample_sub

In [ ]:
train_data_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_data_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

train_df

In [ ]:
def read_json_pub(filename, train_data_path=train_data_path, output='text'):
    json_path = os.path.join(train_data_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    text = re.sub(' +', ' ', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text


In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
#temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
#temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
#temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]

#existing_labels = set(temp_1 + temp_2 + temp_3)
#df2 = pd.DataFrame(list(existing_labels))

#df2.rename(columns = {0:'title'}, inplace = True)

#df2['title']=df2['title'].str.replace(",","")
#df2['title']=df2['title'].str.replace("-"," ")

#df2['length'] = df2['title'].str.len()
#df2.sort_values('title', ascending=True, inplace=True)

#df2.head(50)

In [ ]:
#df2.to_csv('datasets.csv', index = False)

In [ ]:
#df2=pd.read_csv('../input/extradata/datasets.csv')
df = pd.read_csv('../input/bigger-govt-dataset-list/data_set_26897.csv')
#df2 = pd.read_csv('../input/extradata/datasets.csv', sep='delimiter', header=None,encoding= 'unicode_escape')
df.columns = ['title']
df['title']=df['title'].str.replace(" +"," ")
print(len(df))
#f2.sort_values('title', ascending=True, inplace=True)

In [ ]:
df.head(20)

In [ ]:
#df2.head(20)

In [ ]:
# result = df.append(df2)
# print(len(result))

In [ ]:
df = df.drop_duplicates()
print(len(df))

In [ ]:
# random sample for testing
train_sample=train_df.sample(n = 10)

train_sample

In [ ]:
from re import search
start_time = time.time()


#### remove >.5 jaccard matches from predicitons
def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

#############################
#path=train_data_path
path=test_data_path

#for training use train_sample

#for submission use sample_sub

#############

column_names = ["Id", "PredictionString"]

submission = pd.DataFrame(columns = column_names)

to_append=[]
for index, row in sample_sub.iterrows():
    to_append=[row['Id'],'']
    print(to_append)
    large_string = str(read_json_pub(row['Id'],path))
    clean_string=text_cleaning(large_string)
    #print(clean_string)
    for index, row2 in df.iterrows():
        query_string = str(row2['title'])
        #if search('andi', clean_string):
        #print(re.search(query_string[1], clean_string))
        #if query_string !='time' and query_string !='series' and query_string !='population' and query_string !='annual':
        #if query_string in clean_string:
        if fuzz.token_set_ratio(query_string,clean_string) > 0.3:
                #print(query_string)
            if to_append[1]!='' and clean_text(query_string) not in to_append[1]:
                to_append[1]=to_append[1]+'|'+clean_text(query_string)
            if to_append[1]=='':
                to_append[1]=clean_text(query_string)
     
    ###### remove similar jaccard
    #got_label=to_append[1].split('|')
    #filtered=[]
    #filtered_labels = ''
    #for label in sorted(got_label, key=len):
        #label = clean_text(label)
        #if len(filtered) == 0 or all(jaccard_similarity(label, got_label) < 1.0 for got_label in filtered):
            #filtered.append(label)
            #if filtered_labels!='':
                #filtered_labels=filtered_labels+'|'+label
            #if filtered_labels=='':
                #filtered_labels=label
    #to_append[1] = filtered_labels         
    #print ('################')
    #print (to_append)
    #print (large_string)
    #print ('################')
    ###### remove similar jaccard
    df_length = len(submission)
    submission.loc[df_length] = to_append
submission.to_csv('submission.csv', index = False)
print("--- %s seconds ---" % (time.time() - start_time))
to_append